In [ ]:
#COLAB - mount drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:

#IMAGE ANALYSIS FOR STYLEGAN PAINTINGS GENERATED BY CUSTOM TRAINED STYLEGAN2 MODELS
#by JON THUM


!git clone https://github.com/jonthum/AiPainter
%cd AiPainter

#COLAB - root directory
ROOT_DIR = 'AiPainter/'

#COLAB - import .py files
import sys
SG2_DIR = ROOT_DIR + 'lib/stylegan2/'
sys.path.append(SG2_DIR)

#CHECK TF SETTINGS
%tensorflow_version 1.x
import tensorflow as tf
#print('Tensorflow version: {}'.format(tf.__version__) )

#IMPORT LIBRARIES
import dnnlib
import dnnlib.tflib as tflib
import pickle

Gmodels = []

#LIMIT TF MEMORY USE
cfg = dict()
cfg["gpu_options.per_process_gpu_memory_fraction"]=0.2
tflib.init_tf(cfg)

#DOWNLOAD CUSTOM SG2 MODELS
print('Downloading SG2 models ..')
!gdown --id 1-4vOSwOjEn6bRkirUDm9pzPTI1mr10Rb -q
!gdown --id 1-5CurIFM-sEprbytN0bWtlZ8HxXeESXY -q
!gdown --id 1-FJoffvHQffJtlzz6t725PJnbvFQLJdR -q
!gdown --id 1-HJI2gegpAXJGi7odlxKVKvESTGCtadZ -q
!gdown --id 1-K2oUvzk3AphYq1QoymrHoDwyp_JiP5I -q

#LOAD SG2 MODELS
print('Loading SG2 models ..')
with open('networkA_Gs.pkl', 'rb') as f:
    Gs = pickle.load(f)
Gmodels.append(Gs)
print('Loaded model A')

with open('networkB_Gs.pkl', 'rb') as f:
    Gs = pickle.load(f)
Gmodels.append(Gs)
print('Loaded model B')

with open('networkC_Gs.pkl', 'rb') as f:
    Gs = pickle.load(f)
Gmodels.append(Gs)
print('Loaded model C')

with open('networkD_Gs.pkl', 'rb') as f:
    Gs = pickle.load(f)
Gmodels.append(Gs)
print('Loaded model D')

with open('networkE_Gs.pkl', 'rb') as f:
    Gs = pickle.load(f)
Gmodels.append(Gs)
print('Loaded model E')
print('SG2 models loaded')

#Model dictionary
MODEL = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E'}

#IMPORT LIBRARIES
import os
import os.path
import time
import numpy as np
import cv2
from PIL import Image, ImageFilter, ImageStat, ImageChops, ImageMath
import statistics as st




In [ ]:
#IMAGE ANALYSIS AND FACE DETECTION FOR 5 MODELS - SAVE IN NUMPY FILES


#HSV COLOUR HUES  
colour = {'red': 255, 'yellow': 42, 'green': 85, 'cyan': 127, 'blue':170, 'magenta': 212}
centre = 0

#KEY HUE VALUES
def hue_key(x):
    global centre
    radius = 43     #max 127
    
    #Shift everything to origin at 128 to avoid negative numbers    
    y = (x-centre+128)%255       
    key = int(3*(radius - abs(y-128)))
    key = max(key, 0)   
    
    return key

#MEAN VALUE FOR SPECIFIED COLOUR = HUE KEY x SAT x VAL
def colour_stat(hue, sat, val):
    key = Image.eval(hue, hue_key)
    col = ImageMath.eval("convert(a*b/255, 'L')", a=key, b=sat)
    colval = ImageMath.eval("convert(a*b/255, 'L')", a=col, b=val)
    stat = ImageStat.Stat(colval)
    
    return stat.mean[0]


#GENERATE IMAGE FROM MODEL
def generate_painting(seed, artstyle):
    rnd = np.random.RandomState(seed)
    latent = rnd.randn(1, Gmodels[artstyle].input_shape[1]) 
    images = Gmodels[artstyle].run(latent, None, truncation_psi=1.0, randomize_noise=False, output_transform=fmt)
    return images[0]

#MODEL ARGS
fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
Gs_syn_kwargs = dnnlib.EasyDict()
Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
Gs_syn_kwargs.minibatch_size = 1


#INIT
start = time.process_time()
print('Processing ....')
DATA_DIR = ROOT_DIR + 'custom/imageanalysis/'

#FACE DETECT
faceCascade = cv2.CascadeClassifier(DATA_DIR + 'haarcascade_frontalface_default.xml')
MIN_SIZE = 200   #Only detect faces bigger than 200x200

#INCLUSIVE RANGE
START = 0
END = 5000


#LOOP MODELS
for model in range(5):
  
    datafile = []

    #LOOP THROUGH IMAGES
    for id in range(START, END+1):

        #PROGRESS FEEDBACK
        if(id%500 == 0):
            print('{} images:'.format(id), time.process_time() - start)
       
        #GENERATE PAINTING
        painting = generate_painting(id, model)
        image = Image.fromarray(painting)

        #FACE DETECT
        cv2image = np.array(image)
        gray = cv2.cvtColor(cv2image, cv2.COLOR_RGB2GRAY)
        faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, \
                                             minSize=(MIN_SIZE, MIN_SIZE))
        num_faces = len(faces)
        if (num_faces):
            print('{} faces detected id {}'.format(num_faces, id))

        #DETAIL ANALYSIS
        grey = image.convert('L')
        blur = grey.filter(ImageFilter.GaussianBlur(10))
        diff = ImageChops.difference(grey, blur)
        
        D = 4   #Normalising constant
        stat = ImageStat.Stat(diff)
        detail = int(D*stat.mean[0])
        
        
        #DOWNSIZE TO SAVE COMPUTATION
        image = image.resize((128,128))
      
        
        #HSV ANALYSIS
        hsv = image.convert(mode="HSV")
        
        hue = hsv.getchannel('H')
        sat = hsv.getchannel('S')
        val = hsv.getchannel('V')
     
        stat = ImageStat.Stat(hsv)
        brightness = int(stat.mean[2]/2)
        contrast = int(stat.stddev[2])
        saturation = int(stat.mean[1]/2)
        
    
        #INDIVIDUAL COLOUR ANALYSIS
        C = 8   #Normalising constant
        
        centre = colour['red']
        red_mean = int(C*colour_stat(hue, sat, val))
        
        centre = colour['yellow']
        yellow_mean = int(C*colour_stat(hue, sat, val))
        
        centre = colour['green']
        green_mean = int(C*colour_stat(hue, sat, val))
        
        centre = colour['cyan']
        cyan_mean = int(C*colour_stat(hue, sat, val))
        
        centre = colour['blue']
        blue_mean = int(C*colour_stat(hue, sat, val))
        
        centre = colour['magenta']
        magenta_mean = int(C*colour_stat(hue, sat, val))
    
    
        #COLOUR VARIETY
        T = 60  #Max value to determine completeness
        
        col_means = [red_mean, yellow_mean, green_mean, cyan_mean, blue_mean, magenta_mean]
        mean = st.mean(col_means)
        stdev = st.stdev(col_means)
        total = min(red_mean, T) + min(yellow_mean, T) + min(green_mean, T) + \
                min(cyan_mean, T) + min(blue_mean, T) + min(magenta_mean, T)
        variety = int(total*mean/(stdev+1)/2)    
                
        #APPEND DATA
        info = [detail, brightness, contrast, saturation, variety, 
                red_mean, yellow_mean, green_mean, 
                cyan_mean, blue_mean, magenta_mean, num_faces, id]
        #print(info)
        datafile.append(info)        
        

    #SAVE DATA
    datafile = np.array(datafile, dtype=np.int)
    name = 'ImageAnalysis1_{}.npy'.format(MODEL[model])
    np.save(DATA_DIR + name, datafile)
    print(datafile.shape)

    print('END:', time.process_time() - start)










In [ ]:
#PROGRAM TO CORRELATE FACE DETECTION AMONGST MODELS 
#SET ALL TO ONE IF DETECTED IN MORE THAN 2 MODELS, ELSE SET ALL TO ZERO

DATA_DIR = ROOT_DIR + 'custom/imageanalysis/'

#LOAD IMAGE ANALYSIS FILES
datafiles = []
datafiles.append(np.load(ROOT_DIR + 'custom/imageanalysis/ImageAnalysis1_A.npy'))
datafiles.append(np.load(ROOT_DIR + 'custom/imageanalysis/ImageAnalysis1_B.npy'))
datafiles.append(np.load(ROOT_DIR + 'custom/imageanalysis/ImageAnalysis1_C.npy'))
datafiles.append(np.load(ROOT_DIR + 'custom/imageanalysis/ImageAnalysis1_D.npy'))
datafiles.append(np.load(ROOT_DIR + 'custom/imageanalysis/ImageAnalysis1_E.npy'))

num = len(datafiles[0]) #20
print(num)

for i in range(num):

    #CHECK HOW MANY MODELS FOUND FACES
    faces = 0
    for model in range(5):
        faces += datafiles[model][i,11] 
        #print(datafiles[model][i,11], end='')
    #print('= {}'.format(faces))

    #IF MORE THAN 2 FACES DETECTED SET ALL TO 1 
    if (faces>2):
        for model in range(5):
            datafiles[model][i,11] = 1
    #ELSE SET ALL TO 0
    else:
        for model in range(5):
            datafiles[model][i,11] = 0

    #CHECK
    #for model in range(5):
        #print(datafiles[model][i,11], end='')
    #print('')


#SAVE DATA
for model in range(5):
    name = 'ImageAnalysis2_{}.npy'.format(MODEL[model])
    np.save(DATA_DIR + name, datafiles[model])
    print(datafiles[model].shape)







